## **1. Data Input**

In [4]:
import pandas as pd
import numpy as np
valid_path = "/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/dataset/Featurized_data/BM_stratified_sampling/external_test_set/avalon_test.csv"
df_valid = pd.read_csv(valid_path)
df_valid = df_valid[["ID","Standardize_smile"]]
df_valid["Activity"]=0
df_valid.head()

,ID,Standardize_smile,Activity
0,Isonitrile_1,[C-]#[N+]C1=CC(=CCC(=O)O)CC1,0
1,1a,c1ccc(-c2nc3ccccc3n3c(N4CCN(c5ccccc5)CC4)ccc23...,0
2,1b,COc1ccccc1N1CCN(c2ccc3c(-c4ccccc4)nc4ccccc4n23...,0
3,2963625,COc1cc2c(cc1NC(=O)CSc1nnc(Cc3cccs3)n1Cc1ccco1)...,0
4,44201431,C=CCC(CC(=O)N(CCO)Cc1ccccc1)C(=O)NC(COC(=O)C(C...,0


In [2]:
df_valid = df_valid[["ID","Standardize_smile"]]
df_valid.iloc[0:10,:].to_csv("data_deploy.csv",index=False)

## **2. Molecular Representation and Molecular Graph Calculation**

In [5]:
from Utility.Featurizer_streamlit import featurizer
featurizer_st = featurizer(df_valid, ID = "ID", smiles_col = "Standardize_smile", active_col = "Activity", save_dir = "Cdr1_classification/", n_jobs = 5)
rdk5_df, rdk6_df, rdk7_df, avalon_df, ph4_processed, mordred_df, graph_dataset = featurizer_st.run()

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/177 [00:00<?, ?it/s]

100%|██████████| 177/177 [00:11<00:00, 15.54it/s]


Output()

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'


Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


reading NP model ...
reading NP model ...
reading NP model ...
reading NP model ...
model in
reading NP model ...
model in


model in
model in
model in


## **3. Prediction**

In [13]:
# from Utility.cdr1_model_predict import ensemble_predict
fpt_datasets = [rdk5_df, rdk6_df, rdk7_df, mordred_df, avalon_df, ph4_processed]
predict = ensemble_predict(fpt_datasets, graph_dataset, "ID", "Standardize_smile")
df_predictions = predict.predict()
display(df_predictions.head())

/Users/thechuongtrinh/anaconda3/envs/cdr1_inhibitors_classification/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/thechuongtrinh/anaconda3/envs/cdr1_inhibitors_classification/lib/python3.9/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,ID,Standardized SMILES,Probability,Prediction
0,Isonitrile_1,[C-]#[N+]C1=CC(=CCC(=O)O)CC1,0.153331,0
1,1a,c1ccc(-c2nc3ccccc3n3c(N4CCN(c5ccccc5)CC4)ccc23...,0.659775,1
2,1b,COc1ccccc1N1CCN(c2ccc3c(-c4ccccc4)nc4ccccc4n23...,0.631182,1
3,2963625,COc1cc2c(cc1NC(=O)CSc1nnc(Cc3cccs3)n1Cc1ccco1)...,0.003459,0
4,44201431,C=CCC(CC(=O)N(CCO)Cc1ccccc1)C(=O)NC(COC(=O)C(C...,0.144681,0


In [15]:
from sklearn.metrics import f1_score, balanced_accuracy_score,roc_auc_score,average_precision_score
# valid_path = "/Users/thechuongtrinh/Documents/Workspace/Master_thesis/Cdr1_inhibitors/dataset/Featurized_data/BM_stratified_sampling/validation_set_dl/avalon_valid.csv"
df_valid_raw = pd.read_csv(valid_path)
y_true = df_valid_raw["Activity"]
y_pred = df_predictions["Prediction"]
y_prob = df_predictions["Probability"]

f1 = f1_score(y_true, y_pred)
balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)
average_precision = average_precision_score(y_true, y_prob)
print(f"F1 score: {f1}")
print(f"Balanced accuracy: {balanced_accuracy}")
print(f"ROC AUC: {roc_auc}")
print(f"Average precision: {average_precision}")


F1 score: 0.7142857142857143
Balanced accuracy: 0.7987068965517241
ROC AUC: 0.8836206896551725
Average precision: 0.7545476551440111
